In [1]:
import re
import pandas as pd
from sklearn.svm import SVC
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
import keras
from sklearn.metrics import classification_report,confusion_matrix
import nltk
import matplotlib.pyplot as plt
from sklearn.model_selection import (train_test_split,cross_val_predict,
                                     cross_val_score,cross_validate,GridSearchCV,StratifiedKFold
                                    )
from sklearn import ensemble, linear_model,svm,neighbors,neural_network
from sklearn.metrics import (classification_report,confusion_matrix,
                             accuracy_score,auc,mean_absolute_error,r2_score,mean_squared_error,
                             plot_roc_curve,roc_auc_score,roc_curve)
from sklearn.preprocessing import (OneHotEncoder,LabelEncoder,MinMaxScaler,StandardScaler,)
from sklearn.feature_selection import SelectKBest, chi2
from xgboost import XGBClassifier,XGBRegressor
# from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings("ignore")

In [2]:
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer(language='english')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stop_words = {"a","an","the"}
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def remove_html_tags(raw_html):
  return re.sub(r'<.*?>', '', raw_html)

def remove_urls(raw_text):
    raw_text = re.sub(r'http\S+', '', raw_text)
    return re.sub(r'www\S+', '', raw_text)

def remove_nonalpha(raw_text):
  return re.sub(r'[^a-zA-Z\s]', '', raw_text)

def remove_lines(raw_text):
  return re.sub(r'\n',' ',raw_text)

def lemme_stem_stop(raw_text):
  words = word_tokenize(raw_text)
  final_words=[]
  for word in words:
    word = word.lower()
    if word not in stop_words:
      word = snowball_stemmer.stem(word)
      final_words.append(word)
  return " ".join(final_words)

def pre_processing(raw_text):
  return lemme_stem_stop(remove_lines(remove_nonalpha(remove_urls(remove_html_tags(raw_text)))))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
df = pd.read_csv("/content/Final.csv")
df = df[df["bias"] != 2]
# text_data = text_df["text"].map(lambda x: pre_processing(x)) 

In [4]:
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer(language='english')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
sw = stopwords.words('english')
unwanted = {"he", "him", "his", "himself", "she", "she's", "her", "hers", "herself", "they", "them", "their", "theirs", "themselves"}
sw = [el for el in sw if el not in unwanted]

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(sw)

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    # text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df.text = df.text.apply(clean_text)
df.text = df.text.str.replace('\d+', '')

In [6]:
df

,Unnamed: 0,id,text,bias
0,0,1,besides forests naturally developed forests ma...,0
1,1,2,mother cleans rice separating husk mud particl...,0
2,2,3,free gifts nature things man cannot produce in...,0
3,3,4,humanism concept man measure things,0
4,4,5,various designs clothing found depending count...,0
...,...,...,...,...
1991,1991,1992,mulberry soyabean cotton tobacco flax jinseng ...,1
1993,1993,1994,b self employment picture eradicate widespread...,1
1995,1995,1996,bring ends threads together put knot ends threads,1
1997,1997,1998,potassiumprotein synthesisenables opening clos...,1


In [7]:
def apply_model(rmf,x,y,isval=False):
    if not isval:
        rmf.fit(x,y)
    y_pred=rmf.predict(x)
    if isval:
        print("\nValidation:")
    else:
        print("\nTrain:")
    print(classification_report(y,y_pred))
    print()
    print(confusion_matrix(y,y_pred))
    print("-"*50)
    return rmf

In [8]:
import gensim.downloader as api
from gensim.models import Word2Vec
w2v = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [9]:
w2v

In [10]:

s_data = df.text.map(lambda x: x.split())

# model_Word2Vec = Word2Vec(sentences=s_data, size=100, window=5, min_count=1, workers=4,batch_words = 2,sg = 0) #sg =1 for skipgram

# train the model
# model_Word2Vec.train(s_data, total_examples=model_Word2Vec.corpus_count, epochs=model_Word2Vec.iter)


def f3(x):
  vec1 = [w2v[word] for word in x if word in w2v.vocab]
  return vec1  

def fn(x):
  vec = f3(x)
  vec = np.array(vec).mean(axis=0)
  return vec

X = s_data.map( lambda x: fn(x))

y = df["bias"]


X_np=[]
for i in X.keys():
  X_np.append(np.array(X[i]))

X_np=np.array(X_np)
X_train_np, X_test_np, y_train, y_test = train_test_split(X_np, y, test_size=0.1, random_state=42,stratify=y)

In [11]:
# from sklearn.feature_extraction.text import CountVectorizer

# vectorizer1 = CountVectorizer()
# X1 = vectorizer1.fit_transform(text_df["text"])
# y = text_df["bias"]
# X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.1, random_state=42,stratify=y)

rmf = XGBClassifier(iterations=15,verbose=0)
rmf=apply_model(rmf,X_train_np,y_train)
rmf=apply_model(rmf,X_test_np,y_test,isval=True)


Train:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       496
           1       1.00      0.99      1.00       540

    accuracy                           1.00      1036
   macro avg       1.00      1.00      1.00      1036
weighted avg       1.00      1.00      1.00      1036


[[496   0]
 [  3 537]]
--------------------------------------------------

Validation:
              precision    recall  f1-score   support

           0       0.85      0.75      0.80        55
           1       0.79      0.89      0.84        61

    accuracy                           0.82       116
   macro avg       0.82      0.82      0.82       116
weighted avg       0.82      0.82      0.82       116


[[41 14]
 [ 7 54]]
--------------------------------------------------


In [12]:
# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
 
param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 6]
        }

  
grid = GridSearchCV(rmf, param_grid, refit = True, verbose = 3)


# fitting the model for grid search
grid.fit(X_train_np,y_train)

print(grid.best_params_)

rmf_best = grid.best_estimator_

Fitting 5 folds for each of 540 candidates, totalling 2700 fits
[CV 1/5] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.841 total time=   1.0s
[CV 2/5] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.879 total time=   1.0s
[CV 3/5] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.908 total time=   1.0s
[CV 4/5] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.884 total time=   1.0s
[CV 5/5] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.6;, score=0.855 total time=   1.0s
[CV 1/5] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.8;, score=0.832 total time=   1.0s
[CV 2/5] END colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, subsample=0.8;, score=0.870 total time=   1.0s
[CV 3/5] END colsample_bytree=0.6, gamma=0.5, max_dept

In [52]:
rmf_best = XGBClassifier(ierations = 15,colsample_bytree = 0.8, gamma= 1.5, max_depth= 3, min_child_weight= 10, subsample = 0.8)

In [53]:
model=apply_model(rmf_best,X_train_np,y_train)


Train:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       496
           1       1.00      0.98      0.99       540

    accuracy                           0.99      1036
   macro avg       0.99      0.99      0.99      1036
weighted avg       0.99      0.99      0.99      1036


[[494   2]
 [ 11 529]]
--------------------------------------------------


In [35]:
grid.best_params_

NameError: ignored

In [54]:
model_best = apply_model(rmf_best,X_test_np,y_test)


Train:
              precision    recall  f1-score   support

           0       0.95      0.95      0.95        55
           1       0.95      0.95      0.95        61

    accuracy                           0.95       116
   macro avg       0.95      0.95      0.95       116
weighted avg       0.95      0.95      0.95       116


[[52  3]
 [ 3 58]]
--------------------------------------------------


In [44]:
y_hat = rmf_best.predict(X_np)
len(y_hat)

1152

In [45]:
try:
  text_df = df.drop('Unnamed: 0',axis=1)
  text_df = text_df.set_index('id')
except:
  pass
text_df

,text,bias
id,,
1,besides forests naturally developed forests ma...,0
2,mother cleans rice separating husk mud particl...,0
3,free gifts nature things man cannot produce in...,0
4,humanism concept man measure things,0
5,various designs clothing found depending count...,0
...,...,...
1992,mulberry soyabean cotton tobacco flax jinseng ...,1
1994,b self employment picture eradicate widespread...,1
1996,bring ends threads together put knot ends threads,1


In [46]:
text_df['pred'] = y_hat
text_df

,text,bias,pred
id,,,
1,besides forests naturally developed forests ma...,0,1
2,mother cleans rice separating husk mud particl...,0,1
3,free gifts nature things man cannot produce in...,0,0
4,humanism concept man measure things,0,0
5,various designs clothing found depending count...,0,0
...,...,...,...
1992,mulberry soyabean cotton tobacco flax jinseng ...,1,1
1994,b self employment picture eradicate widespread...,1,1
1996,bring ends threads together put knot ends threads,1,1


In [47]:
pair_df = pd.read_csv('/content/pairs.txt',names=['p1','p2','label'])
pair_df

,p1,p2,label
0,98,651,0
1,696,1870,1
2,1073,1620,0
3,167,633,0
4,4,640,0
...,...,...,...
155946,546,1512,1
155947,720,1902,1
155948,1112,1591,0
155949,396,1472,1


In [48]:

pred_label = []
for i in range(pair_df.shape[0]):
    pred_label.append(np.logical_xor(text_df.pred[pair_df.p1[i]],text_df.pred[pair_df.p2[i]]).astype(int))

In [49]:
print(classification_report(pair_df.label,pred_label))

              precision    recall  f1-score   support

           0       0.61      0.60      0.61     80400
           1       0.59      0.60      0.59     75551

    accuracy                           0.60    155951
   macro avg       0.60      0.60      0.60    155951
weighted avg       0.60      0.60      0.60    155951



In [50]:
print(confusion_matrix(pair_df.label,pred_label))

[[48258 32142]
 [30229 45322]]


In [51]:
text_df[text_df.bias!=text_df.pred]

,text,bias,pred
id,,,
1,besides forests naturally developed forests ma...,0,1
2,mother cleans rice separating husk mud particl...,0,1
19,raider back towards corner court defender g si...,0,1
23,iodine exist one three oxides chemist produced...,0,1
28,organism closely related man evolutionary view...,0,1
...,...,...,...
1978,pleasure one work,1,0
1980,exercises four alternatives given complete inc...,1,0
1988,march indian flag hoisted kohima change fortun...,1,0


In [24]:
model.save_model('word2vec-G-xgboost')

In [27]:
model = apply_model(model_best,X_train_np,y_train)


Train:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       496
           1       1.00      0.98      0.99       540

    accuracy                           0.99      1036
   macro avg       0.99      0.99      0.99      1036
weighted avg       0.99      0.99      0.99      1036


[[494   2]
 [ 11 529]]
--------------------------------------------------


In [28]:
model = apply_model(model,X_test_np,y_test,isval=True)


Validation:
              precision    recall  f1-score   support

           0       0.86      0.80      0.83        55
           1       0.83      0.89      0.86        61

    accuracy                           0.84       116
   macro avg       0.85      0.84      0.84       116
weighted avg       0.85      0.84      0.84       116


[[44 11]
 [ 7 54]]
--------------------------------------------------


In [29]:
model = apply_model(model_best,X_np,y)
model = apply_model(model,X_test_np,y_test,isval=True)


Train:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       551
           1       0.99      0.98      0.98       601

    accuracy                           0.98      1152
   macro avg       0.98      0.98      0.98      1152
weighted avg       0.98      0.98      0.98      1152


[[546   5]
 [ 15 586]]
--------------------------------------------------

Validation:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        55
           1       1.00      0.98      0.99        61

    accuracy                           0.99       116
   macro avg       0.99      0.99      0.99       116
weighted avg       0.99      0.99      0.99       116


[[55  0]
 [ 1 60]]
--------------------------------------------------
